# Introducing Single Agents with LangGraph - SOLUTIONS

This notebook contains the complete solution for the Challenge: Add User Preferences.

## Introduction

In this lab, you'll build your first single-agent system using LangGraph. This agent will:

- **Remember conversations** across multiple interactions
- **Maintain context** using LangGraph's state management
- **Persist history** with checkpointers
- **Handle multiple users** through thread IDs

### What is LangGraph?

LangGraph is a framework for building stateful, multi-actor applications with LLMs. It extends LangChain with:

1. **StateGraph**: Directed graph of nodes (functions) and edges (control flow)
2. **Checkpointing**: Automatic state persistence
3. **Streaming**: Real-time output as the graph executes
4. **Thread-based sessions**: Isolated conversation histories

### The Scenario

You'll build a customer support chatbot that:
- Answers questions about a fictional product ("SuperWidget")
- Remembers what the user asked before
- Can be reset to start fresh conversations
- Supports multiple concurrent users via thread IDs

## Setup and Installation

Install LangGraph 1.0+ and LangChain 1.0+:

In [1]:
%pip install -qU langgraph langchain langchain-openai


[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
from typing import Annotated, TypedDict
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langgraph.checkpoint.memory import MemorySaver
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage
import os
import getpass

# API key setup
if "OPENAI_API_KEY" not in os.environ:
    os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API key: ")

print("✓ Environment configured")

✓ Environment configured


## Step 1: Define the Agent's State

In LangGraph, **state** is the data that flows through your graph. We use a `TypedDict` to define what data the agent tracks.

For a conversational agent, we need:
- **messages**: The conversation history

The `Annotated[list, add_messages]` tells LangGraph to **append** new messages rather than replacing the list.

In [3]:
class State(TypedDict):
    """State for customer support chatbot."""
    messages: Annotated[list, add_messages]  # Conversation history

print("✓ State defined")
print("  - messages: list (appends new messages)")

✓ State defined
  - messages: list (appends new messages)


## Step 2: Create the Agent Node

A **node** in LangGraph is a Python function that:
1. Takes the current state as input
2. Does some work (like calling an LLM)
3. Returns a dictionary with state updates

Our `chatbot` node:
- Gets the conversation history from `state["messages"]`
- Adds a system prompt about SuperWidget
- Calls the LLM
- Returns the LLM's response to be added to messages

In [4]:
# Initialize LLM
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.7)

# System prompt with product knowledge
SYSTEM_PROMPT = """You are a helpful customer support agent for SuperWidget, 
a revolutionary smart home device that controls all your appliances with voice commands.

SuperWidget features:
- Voice control for lights, thermostats, locks, and appliances
- Works with Alexa, Google Assistant, and Siri
- Easy 5-minute setup via mobile app
- 24/7 customer support
- Price: $199

Common issues:
- "Can't connect to WiFi" → Check 2.4GHz network, restart device
- "Voice not recognized" → Retrain voice profile in app settings
- "Device offline" → Check power cable and WiFi connection

Be friendly, concise, and helpful. Ask clarifying questions if needed.
"""

def chatbot(state: State) -> dict:
    """Customer support chatbot node."""
    # Build messages with system prompt
    messages = [SystemMessage(content=SYSTEM_PROMPT)] + state["messages"]
    
    # Call LLM
    response = llm.invoke(messages)
    
    # Return state update (new message to append)
    return {"messages": [response]}

print("✓ Chatbot node created")
print(f"  Model: gpt-4o-mini")
print(f"  System prompt: {len(SYSTEM_PROMPT)} characters")

✓ Chatbot node created
  Model: gpt-4o-mini
  System prompt: 633 characters


## Step 3: Build the StateGraph

Now we create a graph with:
1. **One node**: `chatbot` (the function we defined)
2. **Two edges**:
   - START → chatbot (entry point)
   - chatbot → END (exit point)

This creates a simple linear flow:
```
START → chatbot → END
```

In [5]:
# Create graph
graph_builder = StateGraph(State)

# Add node
graph_builder.add_node("chatbot", chatbot)

# Add edges
graph_builder.add_edge(START, "chatbot")
graph_builder.add_edge("chatbot", END)

print("✓ Graph structure defined")
print("  Nodes: chatbot")
print("  Flow: START → chatbot → END")

✓ Graph structure defined
  Nodes: chatbot
  Flow: START → chatbot → END


## Step 4: Enable Persistence with Checkpointing

**Persistence** means the agent remembers previous conversations.

We use a **checkpointer** to automatically save state after each execution:
- `MemorySaver()`: In-memory storage (for development/testing)
- **Production alternatives**: SqliteSaver, PostgresSaver

When you compile the graph with a checkpointer, it:
1. Saves state after each execution
2. Loads previous state when you provide the same `thread_id`
3. Enables conversation history across multiple `invoke()` calls

In [6]:
# Create checkpointer
memory = MemorySaver()

# Compile graph with persistence
agent = graph_builder.compile(checkpointer=memory)

print("✓ Agent compiled with persistence")
print("  Checkpointer: MemorySaver (in-memory)")
print("  Ready to remember conversations!")

✓ Agent compiled with persistence
  Checkpointer: MemorySaver (in-memory)
  Ready to remember conversations!


## Step 5: Test the Agent

Let's have a multi-turn conversation to see persistence in action.

**Key concept**: `thread_id` identifies the conversation session.
- Same `thread_id` = continue existing conversation
- Different `thread_id` = start new conversation

### Helper Function

We'll create a helper to make chatting easier:

In [7]:
def chat(user_message: str, thread_id: str = "default") -> str:
    """
    Send a message and get the agent's response.
    
    Args:
        user_message: What the user says
        thread_id: Conversation session ID (default: "default")
    
    Returns:
        The agent's response text
    """
    # Create config with thread ID
    config = {"configurable": {"thread_id": thread_id}}
    
    # Invoke agent with user message
    result = agent.invoke(
        {"messages": [HumanMessage(content=user_message)]},
        config
    )
    
    # Return last message (agent's response)
    return result["messages"][-1].content

print("✓ Helper function ready")
print("  Usage: chat('your message', thread_id='session1')")

✓ Helper function ready
  Usage: chat('your message', thread_id='session1')


### Test 1: Basic Conversation

In [8]:
# Start conversation
response1 = chat("Hi! What is SuperWidget?", thread_id="user1")
print(f"User: Hi! What is SuperWidget?")
print(f"Agent: {response1}")
print()

User: Hi! What is SuperWidget?
Agent: Hi there! SuperWidget is a revolutionary smart home device that allows you to control all your appliances using voice commands. It works seamlessly with Alexa, Google Assistant, and Siri, making it easy to manage your lights, thermostats, locks, and other appliances with just your voice. The setup is simple and only takes about 5 minutes via our mobile app. If you have any specific questions or need more information, feel free to ask!



### Test 2: Follow-up Question (Tests Persistence)

The agent should remember what "it" refers to (SuperWidget):

In [9]:
# Follow-up (same thread)
response2 = chat("How much does it cost?", thread_id="user1")
print(f"User: How much does it cost?")
print(f"Agent: {response2}")
print()
print("✓ Agent remembered the conversation context!")

User: How much does it cost?
Agent: SuperWidget is priced at $199. If you have any more questions or need assistance with anything else, just let me know!

✓ Agent remembered the conversation context!


### Test 3: New User (Different Thread)

A different `thread_id` starts a fresh conversation:

In [10]:
# Different user (new thread)
response3 = chat("I can't connect to WiFi. Help!", thread_id="user2")
print(f"User: I can't connect to WiFi. Help!")
print(f"Agent: {response3}")
print()
print("✓ New thread = isolated conversation")

User: I can't connect to WiFi. Help!
Agent: I'm here to help! Please check the following:

1. **Network Type**: Ensure you’re trying to connect to a 2.4GHz network, as SuperWidget doesn’t support 5GHz networks.
2. **Restart Device**: Try unplugging SuperWidget, waiting for about 10 seconds, and then plugging it back in.

Can you confirm if you're on a 2.4GHz network? If you're still having trouble, let me know!

✓ New thread = isolated conversation


## Step 6: Inspect Conversation State

You can view the saved state for any thread:

In [11]:
# Get state for user1's conversation
state_user1 = agent.get_state({"configurable": {"thread_id": "user1"}})

print("State for user1:")
print(f"  Total messages: {len(state_user1.values['messages'])}")
print(f"\nConversation history:")
for i, msg in enumerate(state_user1.values['messages']):
    role = "User" if isinstance(msg, HumanMessage) else "Agent"
    content = msg.content[:80] + "..." if len(msg.content) > 80 else msg.content
    print(f"  {i+1}. {role}: {content}")

State for user1:
  Total messages: 4

Conversation history:
  1. User: Hi! What is SuperWidget?
  2. Agent: Hi there! SuperWidget is a revolutionary smart home device that allows you to co...
  3. User: How much does it cost?
  4. Agent: SuperWidget is priced at $199. If you have any more questions or need assistance...


## Challenge: Add User Preferences

**Goal**: Extend the agent to remember user preferences beyond just messages.

**Requirements**:
1. Add a `user_name` field to the `State` TypedDict
2. Create a new node `extract_name` that checks if the user introduced themselves
3. If they did, extract and save their name to `state["user_name"]`
4. Modify the `chatbot` node to greet users by name if known
5. Update the graph flow:
   ```
   START → extract_name → chatbot → END
   ```

**Hints**:
- Use the LLM to extract names: "Did the user introduce themselves? If so, what's their name?"
- Check `if state.get("user_name")` to see if name is set
- Update system prompt to include: `f"The user's name is {user_name}" if user_name else ""`

**Test**:
```python
chat("Hi, I'm Alice!", thread_id="alice")
chat("What's my name?", thread_id="alice")  # Should remember "Alice"
```

In [12]:
# Challenge Solution: Add User Preferences

from typing import Annotated, TypedDict, Optional
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langgraph.checkpoint.memory import MemorySaver
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage

# Step 1: Extend State with user_name field
class StateWithName(TypedDict):
    """Extended state with user name tracking."""
    messages: Annotated[list, add_messages]
    user_name: Optional[str]  # User's name if introduced

print("✓ Extended state defined")
print("  - messages: list (conversation history)")
print("  - user_name: Optional[str] (extracted from conversation)")

✓ Extended state defined
  - messages: list (conversation history)
  - user_name: Optional[str] (extracted from conversation)


In [13]:
# Step 2: Create extract_name node

NAME_EXTRACTION_PROMPT = """Look at the user's last message: "{message}"

Did the user introduce themselves or provide their name?

If YES: Respond with ONLY the name (e.g., "Alice" or "Bob")
If NO: Respond with ONLY the word "NONE"

Examples:
- "Hi, I'm Alice!" → Alice
- "My name is Bob" → Bob
- "Hello" → NONE
- "What is SuperWidget?" → NONE
"""

llm_name_extractor = ChatOpenAI(model="gpt-4o-mini", temperature=0)

def extract_name(state: StateWithName) -> dict:
    """Extract user name from conversation if present."""
    # If name already extracted, skip
    if state.get("user_name"):
        print(f"  Name already known: {state['user_name']}")
        return {}  # No state update
    
    # Get last user message
    user_messages = [msg for msg in state["messages"] if isinstance(msg, HumanMessage)]
    if not user_messages:
        return {}  # No user messages yet
    
    last_message = user_messages[-1].content
    
    # Ask LLM to extract name
    prompt = NAME_EXTRACTION_PROMPT.format(message=last_message)
    response = llm_name_extractor.invoke([SystemMessage(content=prompt)])
    
    extracted = response.content.strip()
    
    # Check if name was found
    if extracted != "NONE" and extracted:
        print(f"  ✓ Extracted name: {extracted}")
        return {"user_name": extracted}
    else:
        print(f"  No name found in message")
        return {}  # No name found

print("✓ Name extraction node created")

✓ Name extraction node created


In [14]:
# Step 3: Update chatbot node to use name

SYSTEM_PROMPT_WITH_NAME = """You are a helpful customer support agent for SuperWidget, 
a revolutionary smart home device that controls all your appliances with voice commands.

{name_section}

SuperWidget features:
- Voice control for lights, thermostats, locks, and appliances
- Works with Alexa, Google Assistant, and Siri
- Easy 5-minute setup via mobile app
- 24/7 customer support
- Price: $199

Common issues:
- "Can't connect to WiFi" → Check 2.4GHz network, restart device
- "Voice not recognized" → Retrain voice profile in app settings
- "Device offline" → Check power cable and WiFi connection

Be friendly, concise, and helpful. Ask clarifying questions if needed.
{greeting_instruction}
"""

llm_chatbot = ChatOpenAI(model="gpt-4o-mini", temperature=0.7)

def chatbot_with_name(state: StateWithName) -> dict:
    """Customer support chatbot that greets by name."""
    # Build personalized system prompt
    user_name = state.get("user_name")
    
    if user_name:
        name_section = f"The user's name is {user_name}."
        greeting_instruction = f"Greet {user_name} by name when appropriate."
    else:
        name_section = "The user has not introduced themselves yet."
        greeting_instruction = ""
    
    system_prompt = SYSTEM_PROMPT_WITH_NAME.format(
        name_section=name_section,
        greeting_instruction=greeting_instruction
    )
    
    # Build messages with system prompt
    messages = [SystemMessage(content=system_prompt)] + state["messages"]
    
    # Call LLM
    response = llm_chatbot.invoke(messages)
    
    # Return state update
    return {"messages": [response]}

print("✓ Updated chatbot node created")
print("  Now greets users by name when known")

✓ Updated chatbot node created
  Now greets users by name when known


In [15]:
# Step 4: Build updated graph with name extraction

# Create graph
graph_builder_with_name = StateGraph(StateWithName)

# Add nodes
graph_builder_with_name.add_node("extract_name", extract_name)
graph_builder_with_name.add_node("chatbot", chatbot_with_name)

# Add edges: START → extract_name → chatbot → END
graph_builder_with_name.add_edge(START, "extract_name")
graph_builder_with_name.add_edge("extract_name", "chatbot")
graph_builder_with_name.add_edge("chatbot", END)

# Compile with persistence
memory_with_name = MemorySaver()
agent_with_name = graph_builder_with_name.compile(checkpointer=memory_with_name)

print("✓ Updated graph compiled with persistence")
print("  Flow: START → extract_name → chatbot → END")
print("  Nodes: extract_name, chatbot")

✓ Updated graph compiled with persistence
  Flow: START → extract_name → chatbot → END
  Nodes: extract_name, chatbot


In [16]:
# Step 5: Updated helper function

def chat_with_name(user_message: str, thread_id: str = "default") -> str:
    """
    Send a message and get the agent's response.
    Now extracts and remembers user names!
    
    Args:
        user_message: What the user says
        thread_id: Conversation session ID
    
    Returns:
        The agent's response text
    """
    config = {"configurable": {"thread_id": thread_id}}
    
    result = agent_with_name.invoke(
        {"messages": [HumanMessage(content=user_message)]},
        config
    )
    
    return result["messages"][-1].content

print("✓ Updated helper function ready")
print("  Usage: chat_with_name('Hi, I\\'m Alice!', thread_id='alice')")

✓ Updated helper function ready
  Usage: chat_with_name('Hi, I\'m Alice!', thread_id='alice')


In [17]:
# Test 1: User introduces themselves

print("Test 1: Name Introduction")
print("=" * 50)

response = chat_with_name("Hi, I'm Alice!", thread_id="alice")
print(f"User: Hi, I'm Alice!")
print(f"Agent: {response}")
print()

Test 1: Name Introduction
  ✓ Extracted name: Alice
User: Hi, I'm Alice!
Agent: Hi Alice! How can I assist you today with your SuperWidget?



In [18]:
# Test 2: Agent remembers the name

print("Test 2: Name Recall")
print("=" * 50)

response = chat_with_name("What's my name?", thread_id="alice")
print(f"User: What's my name?")
print(f"Agent: {response}")
print()
print("✓ Agent remembered the name from previous turn!")

Test 2: Name Recall
  Name already known: Alice
User: What's my name?
Agent: Your name is Alice! How can I help you today?

✓ Agent remembered the name from previous turn!


In [19]:
# Test 3: Different user, different name

print("Test 3: Different User")
print("=" * 50)

response = chat_with_name("Hello, my name is Bob", thread_id="bob")
print(f"User: Hello, my name is Bob")
print(f"Agent: {response}")
print()
print("✓ Different thread = different conversation")

Test 3: Different User
  ✓ Extracted name: Bob
User: Hello, my name is Bob
Agent: Hi Bob! It's great to meet you. How can I assist you today with your SuperWidget?

✓ Different thread = different conversation


In [20]:
# Test 4: Inspect state to see saved name

print("Test 4: State Inspection")
print("=" * 50)

state_alice = agent_with_name.get_state({"configurable": {"thread_id": "alice"}})

print(f"\nState for Alice:")
print(f"  user_name: {state_alice.values.get('user_name')}")
print(f"  Total messages: {len(state_alice.values['messages'])}")

state_bob = agent_with_name.get_state({"configurable": {"thread_id": "bob"}})

print(f"\nState for Bob:")
print(f"  user_name: {state_bob.values.get('user_name')}")
print(f"  Total messages: {len(state_bob.values['messages'])}")

print("\n✓ Each thread maintains separate state!")

Test 4: State Inspection

State for Alice:
  user_name: Alice
  Total messages: 4

State for Bob:
  user_name: Bob
  Total messages: 2

✓ Each thread maintains separate state!



### What We Implemented

1. **Extended State** - Added `user_name: Optional[str]` field to track user identity

2. **Name Extraction Node** - Created `extract_name()` that:
   - Checks if name already known (skip if yes)
   - Analyzes last user message with LLM
   - Extracts name if user introduced themselves
   - Updates state with extracted name

3. **Personalized Chatbot** - Updated `chatbot_with_name()` to:
   - Include user's name in system prompt
   - Greet user by name when appropriate
   - Maintain context of user identity

4. **Updated Graph Flow**:
   ```
   START → extract_name → chatbot → END
   ```

### Key Features Demonstrated

✅ **State Extension** - Added fields to TypedDict for richer context

✅ **Multi-Node Workflow** - Chained processing (extraction → response)

✅ **Conditional State Updates** - Only update name if found

✅ **LLM as Tool** - Used LLM for name extraction task

✅ **Thread Isolation** - Each user's name persists separately

### Design Patterns

**State Management**:
```python
# Check if field exists
if state.get("user_name"):
    # Use existing value
    
# Conditional update
return {"user_name": extracted} if extracted else {}
```

**LLM Prompting**:
```python
# Clear, specific instructions
# Examples for few-shot learning
# Simple output format (name or "NONE")
```

**Graph Sequencing**:
```python
# Preprocessing → Main logic
extract_name → chatbot
```

### Production Considerations

**Current Implementation**:
- ✅ Name extraction works for simple introductions
- ✅ Thread isolation prevents name leaks
- ⚠️ Only checks last message (misses earlier introductions)
- ⚠️ No name validation (could extract nonsense)

**Production Enhancements**:
- Check all messages, not just last one
- Validate extracted names (reasonable length, characters)
- Handle name changes ("Actually, call me Bob")
- Add confirmation ("Nice to meet you, Alice! Is that correct?")
- Support nicknames and multiple name formats

### Next Steps

Try extending this pattern to track:
- **User preferences**: Product interests, notification settings
- **Session metadata**: First message time, message count
- **Context flags**: Is user frustrated, needs escalation?
- **Multi-turn tasks**: Shopping cart, booking process

**Great work!** You've built a personalized conversational agent with LangGraph.

## Summary

In this lab, you learned how to build a stateful single agent with LangGraph:

✅ **State Management** - Used TypedDict with `add_messages` to track conversation history

✅ **Nodes** - Created functions that process state and return updates

✅ **Edges** - Defined control flow with START → node → END

✅ **Persistence** - Enabled checkpointing to remember conversations across sessions

✅ **Thread IDs** - Used thread IDs to isolate conversations for different users

### Key Concepts

**StateGraph Workflow**:
1. Define state schema (TypedDict)
2. Create nodes (functions)
3. Build graph (add nodes + edges)
4. Compile with checkpointer
5. Invoke with config (thread_id)

**When to Use Single Agents**:
- Simple conversational bots
- Customer support with FAQ
- Task-specific assistants (scheduling, summarization)
- Any scenario where one LLM call per turn is sufficient

**Next Steps**:
- **Multi-agent systems**: Coordinate multiple specialized agents
- **Tool use**: Give agents access to external APIs
- **Conditional routing**: Dynamic paths based on state
- **Streaming**: Real-time output during execution